## Setup Environment

In [177]:
from dotenv import load_dotenv
import os

# langsh*t (should find an alternative asap!)

from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings

In [189]:
# get ai provider api endpoints and keys from .env
load_dotenv()
print(os.getenv("FAST_LLM_API_BASE"))

# # Todo: We'll use ollama for now, specify the models down below

# FAST_LLM_MODEL_NAME ="gemma3:12b" # keyword extraction
# CONTEXT_LLM_MODEL_NAME="gemma3:12b" # job description extraction from url
# EMBED_LLM_MODEL_NAME="embeddinggemma:latest" # embeddings
# SMART_LLM_MODEL_NAME="gemma3:12b" # final cv generation

https://api.groq.com/openai/v1


## Get Inputs

In [179]:
# jobPostingUrl="https://www.indeed.com/viewjob?jk=10a05154d8eed5da"

In [180]:
cvPath="examples/cv.pdf"
jobPostingPath = "examples/jobPostingText.txt"
additionalInfoPath = "examples/additionalInfo.txt"

# check if jobPostingUrl is defined
try:
    print(jobPostingUrl)
except:
    jobPostingUrl=""


## Static inputs
Prompts & Template

In [181]:
def load_config_file(filename):
    """Load configuration files from the config directory"""
    config_path = os.path.join('config', filename)
    with open(config_path, 'r', encoding='utf-8') as file:
        return file.read().strip()

cvTemplate = load_config_file('cvTemplate.txt')
cvKeywordsPrompt = load_config_file('cvKeywordsPrompt.txt')
jobKeywordsPrompt = load_config_file('jobKeywordsPrompt.txt')

print("Everything loaded successfully!")

Everything loaded successfully!


## Process Inputs

Handle CV and Additonal Info

In [182]:
# read cv:
loader = PyPDFLoader(cvPath)
pages = []
for page in loader.load():
    pages.append(page)
# join pages into a single string
cvRawText = "\n".join([page.page_content for page in pages])

# read additional info
with open(additionalInfoPath, 'r', encoding='utf-8') as file:
    additionalInfo = file.read().strip()

# combine cv and additional info
if additionalInfo:
    cvText = f"""
------------
CV Raw Text:
------------
{cvRawText}
---------------
Additional Info:
---------------
{additionalInfo}
    """
else:
    cvText = cvRawText
print(cvText)


------------
CV Raw Text:
------------
Ahmed Taha
Fresh Software Engineer
 ahmedtaha1234@gmail.com  +201557528856  Cairo, Egypt  creative-geek.tech  github.com/Creative-Geek
 linkedin.com/in/ahmed-taha-thecg  Exempted
PROFILE
Freshly graduated Software Engineer with hands-on experience in web development, AI integrations & Automation, and
multimedia production. Skilled in React, Nodejs, Flask, and Python, aspires to create dynamic, user-friendly applications. Has
delivered projects from web solutions to AI-driven tools—including an Arabic Handwriting E2E OCR system. Strong in UI/UX
design and committed to crafting efficient, engaging digital experiences.
PROJECTS
Tasky,AI-Powered Todo List 04/2025 – 05/2025
Developed a fullstack todo list app with React, Node.js, and Prisma, focusing on user-friendly design and smooth
animations.
Deployed the client, server, and Postgres database, while enforcing security best practices.
Integrated an AI that turns pasted coworker messages into tasks 

Handle Job Posting

In [183]:
# define a function for url job posting extraction (probably the longest part of this program)
def fetchUrl(url):
    pass
def doRAG(url):
    pass


def extract_job_posting_from_url(url):
    jobPostingHTML = fetchUrl(url)
    jobPostingText = doRAG(jobPostingHTML)
    return jobPostingText


In [184]:
# check if job posting is a url or direct text:

if jobPostingUrl:
    jobPostingText = extract_job_posting_from_url(jobPostingUrl)
else:
    with open(jobPostingPath, 'r', encoding='utf-8') as file:
        jobPostingText = file.read().strip()


## Make Some AI Calls 🤙

First, prepare prompts

In [185]:
# format cv keyword extraction prompt
cvKeywordsPrompt = cvKeywordsPrompt.format(cvText=cvText)
# format job posting keyword extraction prompt
jobKeywordsPrompt = jobKeywordsPrompt.format(jobPostingText=jobPostingText)

In [186]:
print(cvKeywordsPrompt)

You are an expert keyword extraction system specializing in resume and CV analysis. Your task is to extract relevant professional keywords from the provided CV text.

**Instructions:**
1. Extract keywords that are professionally relevant and valuable for job matching, skill assessment, or career analysis
2. Focus on the following categories:
   - Technical skills (programming languages, software, tools, frameworks)
   - Professional skills (project management, leadership, analysis, etc.)
   - Industry terms and domain expertise
   - Job titles and roles
   - Certifications and qualifications
   - Relevant technologies and methodologies
   - Important action verbs that demonstrate capabilities

**Guidelines:**
- Extract single words or short phrases (2-3 words maximum)
- Include both explicit skills mentioned and implied competencies
- Normalize variations (e.g., "JavaScript" and "JS" should be "JavaScript")
- Include both technical and soft skills
- Avoid generic words like "the", "and

012 😄

...I mean, hit the api endpoints

In [ ]:
FAST_LLM = ChatOpenAI(model=os.getenv("FAST_LLM_MODEL_NAME"),
                      base_url=os.getenv("FAST_LLM_API_BASE"),
                      api_key=os.getenv("FAST_LLM_API_KEY"),
                      ).with_structured_output(list)

cvKeywordsMessages = [(
    "human",
    cvKeywordsPrompt,
)]

jobKeywordsMessages = [(
    "human",
    jobKeywordsPrompt,
)]

cvKeywords = FAST_LLM.invoke(cvKeywordsMessages)
print(cvKeywords['iterable'])
jobKeywords = FAST_LLM.invoke(jobKeywordsMessages)
print(jobKeywords['iterable'])

# join them as a single string
cvKeywords = ", ".join(cvKeywords['iterable'])
jobKeywords = ", ".join(jobKeywords['iterable'])

print(cvKeywords)
print(jobKeywords)

In [ ]:
# # Make the same AI call 3 times and print results separated by a line
# responses = []
# for _ in range(3):
#     resp = FAST_LLM.invoke(cvKeywordsMessages)
#     responses.append(resp.content.strip())

# print("\n------\n".join(responses))

{"keywords": [
  "Software Engineer",
  "Web Development",
  "AI Integrations",
  "Automation",
  "Multimedia Production",
  "React",
  "Node.js",
  "Flask",
  "Python",
  "UI/UX Design",
  "Arabic Handwriting",
  "OCR",
  "E2E",
  "Head-only",
  "SSR",
  "SEO",
  "CNN",
  "Bi-LSTM",
  "Image Processing",
  "TensorFlow",
  "Google Cloud",
  "Azure",
  "Docker",
  "GitHub",
  "Git",
  "Jira",
  "Linux",
  "Prisma",
  "SQLite",
  "PostgreSQL",
  "MongoDB",
  "Godot Engine",
  "QT5",
  "Graphic Design",
  "Video Editing",
  "Motion Graphics",
  "Adobe Creative Suite",
  "Technical Writing",
  "Content Creation",
  "Communication",
  "Teamwork",
  "Problem-Solving",
  "Adaptability",
  "Creativity",
  "Time Management",
  "Detail-Oriented",
  "Software Developer",
  "IT Specialist",
  "Excel",
  "Autocad",
  "WordPress",
  "E-Commerce",
  "Computer Repairs",
  "Technical Issues",
  "LLMs",
  "Agent AIs",
  "LangChain",
  "Stable Diffusion",
  "Flux",
  "Vertex AI Platform",
  "Computer Eng